In [35]:
import os
import numpy as np
import torch
import pandas as pd
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch, periodogram
from sklearn.preprocessing import StandardScaler
from scipy.stats import kurtosis
from torch.utils.data import random_split
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)

Using Device: cuda


In [36]:
# Noise Filters
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = fs / 2.0
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut=1.0, highcut=200.0, fs=1000.0, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return filtfilt(b, a, data, axis=0)

# Apply after bandpass
def notch_filter(data, freq=60.0, fs=1000.0, quality=30.0):
    b, a = iirnotch(freq, quality, fs)
    return filtfilt(b, a, data, axis=0)

# Noise Metrics for evaluation
def compute_rmse(true, estimate):
    return np.sqrt(np.mean((true - estimate) ** 2))

def compute_spectral_flatness_band(signal, fs, band):
    f, Pxx = periodogram(signal, fs=fs)
    idx = (f >=band[0]) & (f <=band[1])
    Pxx = Pxx[idx] + 1e-12
    gm = np.exp(np.mean(np.log(Pxx)))
    am = np.mean(Pxx)
    return gm / am

mu_band = (8, 13)
beta_band = (13, 30)
hg_band = (70, 200)

# Kurtosis signal reduction > 0 shows a denoised signal
def proportion_of_positive_kurtosis_signals(kurtosis_raw, kurtosis_denoised):
    return (np.array([(kurtosis_raw - kurtosis_denoised) > 0]).sum() / len(kurtosis_raw)) * 100

# Use a Standard scaler to reduce the mean to 0 and std to 1

In [37]:
# Model definitions
class EcogMotionDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
# CNN
class EcogToMotionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size = 3, padding = 1)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, 3)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.squeeze(-1)
        x = self.fc(x)
        return x

# Linear Model
class LinearEcogToMotionNet(nn.Module):
    def __init__(self, input_channels = 64, sequence_length = 20, output_dim = 3):
        super().__init__()
        self.linear = nn.Linear(input_channels * sequence_length, output_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = x.flatten(start_dim=1)
        x = self.linear(x)
        return x

# LSTM
class EcogLSTM(nn.Module):
    def __init__(self, input_size = 64, hidden_size = 128, num_layers = 1, output_size = 3):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (h_n, c_n) = self.lstm(x) # lstm_out shape: (batch_size, seq_len, hidden_size)
        last_output = lstm_out[:, -1, :]
        output = self.fc(last_output)
        return output

In [38]:
class PreprocessData:
    def __init__(self, ecog_file_path, motion_file_path):
        self.ecog_file_path = ecog_file_path
        self.motion_file_path = motion_file_path
        self.ecog_data = None
        self.motion_data = None
        self.filtered_ecog = None
        self.scaled_ecog = None
        self.X = None
        self.y = None

    def process(self, eval=False, window_size=20, duration_limit=900):
        self.read_data()
        self.filter_signal(eval=eval)
        self.format_data(window_size=window_size, duration_limit=duration_limit)
        return self.X, self.y
    
    def read_data(self):
        self.ecog_data = pd.read_csv(self.ecog_file_path)
        self.motion_data = pd.read_csv(self.motion_file_path)
        return self

    def filter_signal(self, eval=False):
        ecog_raw = self.ecog_data[self.ecog_data.columns[1:-1]].values

        # Apply filters
        filtered = bandpass_filter(ecog_raw, lowcut=1.0, highcut=200.0, fs=1000.0, order=4)
        denoised = notch_filter(filtered, freq=60, fs=1000.0)

        # Evaluate filters
        if eval:
            kurt_raw = kurtosis(ecog_raw, axis=0, fisher=True)
            kurt_denoised = kurtosis(denoised, axis=0, fisher=True)
            proportion_of_positive_kurtosis_signals(kurt_raw, kurt_denoised)
            compute_rmse(ecog_raw, denoised)

        # Scale
        scaler = StandardScaler()
        self.scaled_ecog = scaler.fit_transform(denoised)

        # Replace in DataFrame
        self.ecog_data = self.ecog_data.copy()
        self.ecog_data[self.ecog_data.columns[1:-1]] = self.scaled_ecog

        # Clean memory
        del ecog_raw, filtered, denoised
        gc.collect()
        return self

    def format_data(self, window_size=20, duration_limit=900):
        ecog_df = self.ecog_data[self.ecog_data["Time"] <= duration_limit]
        motion_df = self.motion_data[self.motion_data["Motion_time"] <= duration_limit]

        ecog_values = ecog_df[ecog_df.columns[1:-1]].values
        motion_values = motion_df[motion_df.columns[2:]].values

        num_windows = motion_values.shape[0]

        # Create sliding windows
        self.X = np.stack([
            ecog_values[i*window_size:(i+1)*window_size]
            for i in range(num_windows)
        ])
        self.y = motion_values[:num_windows]
        
        # Clean up
        del ecog_values, motion_values
        gc.collect()

    def save(self):
        output_file_path_base = self.ecog_file_path.strip("ecog_data.csv")
        np.save(output_file_path_base + "X.npy", self.X)
        np.save(output_file_path_base + "y.npy", self.y)


In [39]:
os.path.join(os.getcwd(), "src/", "motor_cortex/data/")

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/'

In [40]:
motion_data_file_l = glob(os.path.join(os.getcwd(), "src/", "motor_cortex/data/data/", "**", "motion*.csv"), recursive=True)
ecog_data_file_l = glob(os.path.join(os.getcwd(), "src/", "motor_cortex/data/data/", "**", "ecog*.csv"), recursive=True)

# Reading only the Left Wrist data and training to detect depending on the Left Wrist

In [10]:
motion_data_file_l

['/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-07-12_(S1)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-10-04_(S6)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-09-27_(S5)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-07-19_(S2)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-08-14_(S4)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-07-26_(S3)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-24_(S10)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S1)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-06_(S6)/motion_data.csv',
 '/home/linux-pc/gh/C

In [11]:
motion_data_file_l[0]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-07-12_(S1)/motion_data.csv'

In [10]:
motion_data_file_l[6]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/motion_data.csv'

In [11]:
motion_data_file_l[16]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-03-15_(S1)/motion_data.csv'

In [12]:
ecog_data_file_l[0]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-07-12_(S1)/ecog_data.csv'

In [13]:
ecog_data_file_l[6]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/ecog_data.csv'

In [14]:
ecog_data_file_l[16]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-03-15_(S1)/ecog_data.csv'

## Bilateral Data

In [ ]:
motion_data_file_l[0]

In [ ]:
motion_data_file = motion_data_file_l[0]

In [ ]:
ecog_data_file = ecog_data_file_l[0]

In [ ]:
motion_data_file

In [ ]:
ecog_data_file

In [ ]:
motion_data_bilateral_2018_07_12_S1 = pd.read_csv(motion_data_file)

In [ ]:
motion_data_bilateral_2018_07_12_S1["Motion_time"]

In [ ]:
ecog_data_bilateral_2018_07_12_S1 = pd.read_csv(ecog_data_file)
channel_data = ecog_data_bilateral_2018_07_12_S1.columns[1:-1].values
ecog_data_bilateral_2018_07_12_S1[channel_data].values

In [ ]:
ecog_data_bilateral_2018_07_12_S1["Time"]

In [ ]:
plt.plot(ecog_data_bilateral_2018_07_12_S1["Time"], ecog_data_bilateral_2018_07_12_S1[channel_data].values)

In [ ]:
ecog_channels = ecog_data_bilateral_2018_07_12_S1.columns[1:-1]

In [ ]:
ecog_data_bilateral_2018_07_12_S1[ecog_channels].values

In [ ]:
motion_data_bilateral_2018_07_12_S1_left_wrist = motion_data_bilateral_2018_07_12_S1[motion_data_bilateral_2018_07_12_S1.columns[1:4]]

In [ ]:
motion_data_bilateral_2018_07_12_S1_left_wrist

In [ ]:
motion_data_bilateral_2018_07_12_S1_left_wrist

In [ ]:
ecog_data_bilateral_2018_07_12_S1[channel_data]

In [ ]:
del ecog_data_bilateral_2018_07_12_S1

## Ipsilateral Data (Right Wrist)

In [23]:
current_ecog_data_file = ecog_data_file_l[6]
current_motion_data_file = motion_data_file_l[6]
preprocessor = PreprocessData(current_ecog_data_file, current_motion_data_file)
X, y = preprocessor.process()
preprocessor.save()

In [30]:
# Preprocess Ipsilateral Data
for index in range(6, 16):
    preprocessor = PreprocessData(ecog_data_file_l[index], motion_data_file_l[index])
    X, y = preprocessor.process()
    preprocessor.save()

KeyboardInterrupt: 

In [ ]:
ecog_data_file_l[16:ecog_data_file_l.__len__()]

['/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-03-15_(S1)/ecog_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-04-12_(S3)/ecog_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-04-08_(S2)/ecog_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-04-22_(S7)/ecog_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-04-12_(S4)/ecog_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-04-22_(S9)/ecog_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-04-12_(S5)/ecog_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-04-22_(S8)/ecog_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-05-31_(S10)/ecog_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-04-15_(S6)/ecog_data.csv']

In [ ]:
# # Preprocess Contralateral Data
# for index in range(16, ecog_data_file_l.__len__()):
#     preprocessor = PreprocessData(ecog_data_file_l[index], motion_data_file_l[index])
#     X, y = preprocessor.process()
#     preprocessor.save()

In [44]:
# Read in the data
processed_data_l = glob(os.path.join('/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/', '**', "**", "*.npy"))


In [46]:
processed_data_l[0]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/X.npy'

In [47]:
processed_data_l[1]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/y.npy'

In [49]:
len(processed_data_l) // 2

10

In [ ]:
# # Perform K-Fold Cross Validation
# iterator = iter(processed_data_l)

# for X, y in zip(iterator, iterator):
#     print(X)
#     print(y)

/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/X.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/y.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-24_(S10)/X.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-24_(S10)/y.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S1)/X.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S1)/y.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-06_(S6)/X.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-06_(S6)/y.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S5)/X.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S5)/y.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S3)/X.npy
/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/

In [ ]:
# Creating Train and Validation Sets
dataset = EcogMotionDataset(X, y)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)


In [ ]:
# Defining the model

# Train Linear Model
input_channels = X.shape[2]
sequence_length = X.shape[1]
model = LinearEcogToMotionNet(input_channels, sequence_length)

# Train 1D CNN
# model = EcogToMotionNet()

# Train LSTM
# model = EcogLSTM(input_size=64, hidden_size=128, num_layers=1, output_size=3)


In [ ]:
# Linear Model 

# Train Linear Model
input_channels = X.shape[2]
sequence_length = X.shape[1]
model = LinearEcogToMotionNet(input_channels, sequence_length)

model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(20):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            val_loss += loss.item()
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val.Loss: {val_loss:.4f}")

In [ ]:
# Train 1D CNN
model = EcogToMotionNet()

model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(20):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            val_loss += loss.item()
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val.Loss: {val_loss:.4f}")

In [ ]:
model = EcogLSTM(input_size=64, hidden_size=128, num_layers=1, output_size=3)
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(20):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            val_loss += loss.item()
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val.Loss: {val_loss:.4f}")

In [ ]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os

def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=20, early_stopping_patience=5, model_name="model"):
    model.to(device)
    best_val_loss = float('inf')
    best_epoch = 0
    
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * X_batch.size(0)
        avg_train_loss = running_train_loss / len(train_loader.dataset)
        train_losses.append(avg_train_loss)
        
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                preds = model(X_batch)
                loss = criterion(preds, y_batch)
                running_val_loss += loss.item() * X_batch.size(0)
        avg_val_loss = running_val_loss / len(val_loader.dataset)
        val_losses.append(avg_val_loss)
        
        print(f"{model_name} Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")
        
        # Save best model checkpoint
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_epoch = epoch
            torch.save(model.state_dict(), f"{model_name}_best.pth")
        
        # Early stopping
        if epoch - best_epoch >= early_stopping_patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    return train_losses, val_losses

def plot_losses(losses_dict):
    plt.figure(figsize=(10,6))
    for model_name, (train_losses, val_losses) in losses_dict.items():
        plt.plot(train_losses, label=f"{model_name} Train")
        plt.plot(val_losses, label=f"{model_name} Val")
    plt.xlabel("Epoch")
    plt.ylabel("Loss (MSE)")
    plt.title("Training and Validation Loss Curves")
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming train_loader, val_loader, criterion are defined

# 1. LSTM
lstm_model = EcogLSTM(input_size=64, hidden_size=128, num_layers=1, output_size=3)
lstm_optimizer = torch.optim.Adam(lstm_model.parameters(), lr=1e-3)
lstm_train_losses, lstm_val_losses = train_model(lstm_model, train_loader, val_loader, criterion, lstm_optimizer, device, epochs=20, model_name="LSTM")

# 2. CNN
cnn_model = EcogToMotionNet()
cnn_optimizer = torch.optim.Adam(cnn_model.parameters(), lr=1e-3)
cnn_train_losses, cnn_val_losses = train_model(cnn_model, train_loader, val_loader, criterion, cnn_optimizer, device, epochs=20, model_name="CNN")

# 3. Linear
input_channels = X.shape[2]
sequence_length = X.shape[1]
linear_model = LinearEcogToMotionNet(input_channels, sequence_length)
linear_optimizer = torch.optim.Adam(linear_model.parameters(), lr=1e-3)
linear_train_losses, linear_val_losses = train_model(linear_model, train_loader, val_loader, criterion, linear_optimizer, device, epochs=20, model_name="Linear")

# Plot all losses together
plot_losses({
    "LSTM": (lstm_train_losses, lstm_val_losses),
    "CNN": (cnn_train_losses, cnn_val_losses),
    "Linear": (linear_train_losses, linear_val_losses)
})
